In [1]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    FormulaTableType, FormulaTableTypeSimple, FormulaTableTypeCalculus, FormulaFile)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)

In [22]:
class House:

	def __init__(self, price):
		self._price = price

	@property
	def price(self):
		return self._price
	
	@price.setter
	def price(self, new_price):
		if new_price > 0 and isinstance(new_price, float):
			self._price = new_price
		else:
			print("Please enter a valid price")

	@price.deleter
	def price(self):
		del self._price
		

my_house = House(100)

my_house.price

100

In [5]:
from maths_objects import DataSource, Formulas
data_source = DataSource()
formulas_by_year = Formulas(data_source.formulas_by_year)

formulas_by_year.proofs_required_items

['$T_n = a + (n-1)d$',
 '$S_n = \\dfrac{n}{2}[2a + (n-1)d] = \\dfrac{n}{2}(a+l)$',
 '$T_n = ar^{n-1}$',
 '$S_n = \\dfrac{a(1-r^n)}{1-r} = \\dfrac{a(r^n-1)}{r-1},\\ \\  r \\neq 1$',
 '$S=\\dfrac{a}{1-r},\\ \\ |r|<1$']